## Accès à la base de donnée
En l'absence d'une API dédiée, mais grâce à la disponibilité d'un lien URL stable sur la plateforme data.gouv, nous développons un script Python pour automatiser le téléchargement et l'extraction des données géographiques du réseau d'éclairage public géré par le SIGERLy (Syndicat de Gestion des Énergies de la Région Lyonnaise). Ces données sont fournies sous forme d'un fichier compressé (ZIP) contenant un fichier exploitable shapefiles.

In [1]:
!pip install py7zr geopandas openpyxl tqdm s3fs
!pip install PyYAML xlrd
!pip install git+https://github.com/inseefrlab/cartiflette
import requests
import zipfile
import io
import os
import geopandas as gpd
from cartiflette import carti_download
import folium
from folium import PolyLine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 4.0 MB/s eta 0:00:00a 0:00:01
  Cloning https://github.com/inseefrlab/cartiflette to /tmp/pip-req-build-cknc35fw
  Running command git clone --filter=blob:none --quiet https://github.com/inseefrlab/cartiflette /tmp/pip-req-build-cknc35fw
^C
ERROR: Operation cancelled by user


ModuleNotFoundError: No module named 'cartiflette'

In [2]:
# URL du fichier ZIP
url = "https://www.data.gouv.fr/fr/datasets/r/921760e5-75dd-4691-b8e6-3e7f0bfb22ce"
extraction_name = "ScratchPad_EclairageLyon"
response = requests.get(url)

if response.status_code == 200:
    print("Téléchargement réussi. Extraction des fichiers...")
    
    # Extraction du fichier ZIP
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(extraction_name)
    
    print(f"Fichiers extraits dans le répertoire '{extraction_name}'.")
    
    # Traitement des fichiers extraits
    for root, dirs, files in os.walk(extraction_name):
        for file in files:
            file_path = os.path.join(root, file)
            print(f"Traitement du fichier : {file_path}")
            
            # Traitement fichier shapefile
            if file.endswith(".shp"):
                print(f"Lecture du fichier SHP : {file_path}")
                gdf_eclairage_lyon = gpd.read_file(file_path)
                print("Résumé des données géospatiales :")
                print(gdf_eclairage_lyon.head())
                print(gdf_eclairage_lyon.info())
else:
    print(f"Échec du téléchargement : {response.status_code}")

gdf_eclairage_lyon

Téléchargement réussi. Extraction des fichiers...
Fichiers extraits dans le répertoire 'ScratchPad_EclairageLyon'.
Traitement du fichier : ScratchPad_EclairageLyon/fr-200058493-reseau-eclairage-exterieur-SIGERLy-20230831.qmd
Traitement du fichier : ScratchPad_EclairageLyon/fr-200058493-reseau-eclairage-exterieur-SIGERLy-20230831.prj
Traitement du fichier : ScratchPad_EclairageLyon/fr-200058493-reseau-eclairage-exterieur-SIGERLy-20230831.shx
Traitement du fichier : ScratchPad_EclairageLyon/fr-200058493-reseau-eclairage-exterieur-SIGERLy-20230831.dbf
Traitement du fichier : ScratchPad_EclairageLyon/fr-200058493-reseau-eclairage-exterieur-SIGERLy-20230831.shp
Lecture du fichier SHP : ScratchPad_EclairageLyon/fr-200058493-reseau-eclairage-exterieur-SIGERLy-20230831.shp
Résumé des données géospatiales :
      gid                insee    cablepos  \
0  147167  La Tour-de-Salvagny  souterrain   
1  147166            Vernaison  souterrain   
2  147165            Vernaison  souterrain   
3  147

,gid,insee,cablepos,geometry
0,147167,La Tour-de-Salvagny,souterrain,"LINESTRING (524969.856 5751232.952, 524969.436..."
1,147166,Vernaison,souterrain,"LINESTRING (536120.976 5724140.334, 536119.264..."
2,147165,Vernaison,souterrain,"LINESTRING (536120.976 5724140.334, 536118.329..."
3,147164,Vernaison,souterrain,"LINESTRING (536083.534 5724156.378, 536082.738..."
4,147162,Vernaison,souterrain,"LINESTRING (536120.976 5724140.334, 536118.647..."
...,...,...,...,...
136665,56,Limonest,souterrain,"LINESTRING (531007.606 5750396.951, 531006.95 ..."
136666,56,Limonest,souterrain,"LINESTRING (531006.95 5750396.577, 531006.222 ..."
136667,56,Limonest,souterrain,"LINESTRING (531006.222 5750396.277, 531005.442..."
136668,55,Limonest,souterrain,"LINESTRING (531011.205 5750398.362, 531010.537..."


In [ ]:
# Affichage des communes
valeurs_uniques = gdf_eclairage_lyon['insee'].unique()
print("Valeurs uniques dans la colonne 'cablepos' :")
print(valeurs_uniques)

In [ ]:
#Importation de la carte des arrondissement et création du fond de carte

com_borders_lyon = carti_download(
    crs=4326,
    values=["69"],
    borders="COMMUNE_ARRONDISSEMENT",
    vectorfile_format="geojson",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022,
)

#création du fond de carte
base = com_borders_lyon.plot(color="white", edgecolor="black")
base.set_axis_off()
base.set_title("Répartitions des points lumineux à Lyon")

# Conversion du CRS pour cadrer avec le fond de carte cartiflette
gdf_eclairage_lyon = gdf_eclairage_lyon.to_crs(epsg=4326)


#Intégration de la localisation des caméras de surveillance
gdf_eclairage_lyon.plot(ax=base, color = "red", linewidth=2)

In [ ]:
#Le code s'exécute en c. 4' 
# Création d'une carte Folium centrée sur Lyon (on centre sur la place Bellecour)
map_eclairage_lyon = folium.Map(location=[45.7561, 4.8343], zoom_start=12)

# Contours des arrondissements
folium.GeoJson(
    com_borders_lyon,
    name="communes de la région lyonnaise",
    style_function=lambda feature: {
        'fillColor': 'white',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.5,
    }
).add_to(map_eclairage_lyon)

#Ajout des éclairages
for _, row in gdf_eclairage_lyon.head(100).iterrows():
    tooltip_text = (
        f"Implantation: {row['insee']}<br>type: {row['cablepos']}" 
    )
    coordinates = [(point[1], point[0]) for point in row.geometry.coords]
    # Création de ligne PolyLine
    PolyLine(
        locations=coordinates,
        color='red',
        weight=2,
        opacity=0.8,
        tooltip=tooltip_text
    ).add_to(map_eclairage_lyon)

folium.LayerControl().add_to(map_eclairage_lyon)
    
map_eclairage_lyon.save("carte_videoprotection_paris.html")
map_eclairage_lyon